## **Install Necessary Packages**
 - Always run this when starting the notebook

In [1]:
from netCDF4 import Dataset, num2date, date2num
import glob
import xarray as xr 
import seaborn as sns
from wrf import (to_np, getvar, smooth2d, get_cartopy, cartopy_xlim, vertcross,
                 cartopy_ylim, interplevel, latlon_coords, uvmet, CoordPair,
                ALL_TIMES)
from matplotlib.dates import MonthLocator
import datetime
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import cartopy.feature as cfeature
import cartopy.crs as ccrs
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patheffects as PathEffects
import matplotlib.ticker as mticker
from matplotlib.cm import get_cmap
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

SNOWIE_Data_Path = r'/glade/u/home/mwarms/SNOWIE_Seasonal_Data/wrfout_d02'
Saved_Data_Path =  r'/glade/u/home/mwarms/SavedData/'


## **Create the wrflist**
- This code grabs the 5088 hourly datasets from the Casper storage directory [/glade/u/home/mwarms/SNOWIE_Seasonal_Data/wrfout_d02] and creates a list of 5088 NetCDF files for use with the "wrf.getvar" statement

In [2]:
# ncfile = Dataset(SNOWIE_Data_Path + "/wrfout_d02_2017-01-19_17:00:00")

In [2]:
allfiles = glob.glob(SNOWIE_Data_Path + "/wrfout_d02*")
allfiles.sort()

wrflist = []
for f in range(0,len(allfiles)):
    wrflist.append(Dataset(allfiles[f]))

In [3]:
allfiles[0:4], allfiles[-4:]

(['/glade/u/home/mwarms/SNOWIE_Seasonal_Data/wrfout_d02/wrfout_d02_2016-10-01_00:00:00',
  '/glade/u/home/mwarms/SNOWIE_Seasonal_Data/wrfout_d02/wrfout_d02_2016-10-01_01:00:00',
  '/glade/u/home/mwarms/SNOWIE_Seasonal_Data/wrfout_d02/wrfout_d02_2016-10-01_02:00:00',
  '/glade/u/home/mwarms/SNOWIE_Seasonal_Data/wrfout_d02/wrfout_d02_2016-10-01_03:00:00'],
 ['/glade/u/home/mwarms/SNOWIE_Seasonal_Data/wrfout_d02/wrfout_d02_2017-04-30_20:00:00',
  '/glade/u/home/mwarms/SNOWIE_Seasonal_Data/wrfout_d02/wrfout_d02_2017-04-30_21:00:00',
  '/glade/u/home/mwarms/SNOWIE_Seasonal_Data/wrfout_d02/wrfout_d02_2017-04-30_22:00:00',
  '/glade/u/home/mwarms/SNOWIE_Seasonal_Data/wrfout_d02/wrfout_d02_2017-04-30_23:00:00'])

In [4]:
# Save big files to disk then delete for memory purposes
Saved_Data_Path = r'/glade/scratch/mwarms/temp/'

def write_xarray_to_netcdf(xarray_array, output_path,mode='w', format='NETCDF4', group=None, engine=None,
                           encoding=None):
    """writes and xarray in a netcdf format outputfile
    Uses the xarray typical for wrf-python. The projection objects are transformed into strings
    to be able to use them as netcdf attributes
    :param xarray_array: xarray.DataArray
    :param output_path: str
    :param format: 'NETCDF4', 'NETCDF4_CLASSIC', 'NETCDF3_64BIT' or 'NETCDF3_CLASSIC'
                    default: 'NETCDF4'
    :param group: str, default None
    :param engine: 'netcdf4', 'scipy' or 'h5netcdf'
    :param encoding: dict, default: None
    """
    xarray_array_out = xarray_array.copy(deep=True)
    # coordinates are extracted from variable
    del xarray_array_out.attrs['coordinates']
    # wrf-python projection object cannot be processed
    xarray_array_out.attrs['projection'] = str(xarray_array_out.attrs['projection'])

    xarray_array_out.to_netcdf(path=output_path, mode=mode, format=format, group=group,
                               engine=engine,
                               encoding=encoding)

## Grab SNOWNC and Store to Local Saved Data

In [ ]:
SNOWNC_snowie = getvar(wrflist, "SNOWNC", timeidx=ALL_TIMES, method="cat") # accum snow 

In [ ]:
dummy_file = Dataset(SNOWIE_Data_Path + '/wrfout_d02_2016-10-01_00:00:00')
cartopy_dom = getvar(dummy_file, "ter")

lat_min = 43
lat_max = 46
lon_min = -117
lon_max = -111

mask_lon = (cartopy_dom.XLONG >= lon_min) & (cartopy_dom.XLONG <= lon_max)
mask_lat = (cartopy_dom.XLAT >= lat_min) & (cartopy_dom.XLAT <= lat_max)

del dummy_file, cartopy_dom

In [ ]:
SNOWNC_snowie = SNOWNC_snowie.where(mask_lon & mask_lat, drop=True)

In [ ]:
write_xarray_to_netcdf(SNOWNC_snowie, Saved_Data_Path + "SNOWNC_snowie.nc", 
                       mode = 'w', format = 'NETCDF4', group=None, engine=None, encoding=None)

## Grab RAINNC and Store to Local Saved Data

In [6]:
RAINNC_snowie = getvar(wrflist, "RAINNC", timeidx=ALL_TIMES, method="cat") # accum rain

In [7]:
dummy_file = Dataset(SNOWIE_Data_Path + '/wrfout_d02_2016-10-01_00:00:00')
cartopy_dom = getvar(dummy_file, "ter")

lat_min = 43
lat_max = 46
lon_min = -117
lon_max = -111

mask_lon = (cartopy_dom.XLONG >= lon_min) & (cartopy_dom.XLONG <= lon_max)
mask_lat = (cartopy_dom.XLAT >= lat_min) & (cartopy_dom.XLAT <= lat_max)

del dummy_file, cartopy_dom

In [8]:
RAINNC_snowie = RAINNC_snowie.where(mask_lon & mask_lat, drop=True)

In [10]:
Saved_Data_Path =  r'/glade/u/home/mwarms/SavedData/'

write_xarray_to_netcdf(RAINNC_snowie, Saved_Data_Path + "RAINNC_snowie.nc", 
                       mode = 'w', format = 'NETCDF4', group=None, engine=None, encoding=None)

## **Grab QCLOUD Data Month by Month -- Store to Scratch**

**Load Mask Info**

In [ ]:
dummy_file = Dataset(SNOWIE_Data_Path + '/wrfout_d02_2016-10-01_00:00:00')
cartopy_dom = getvar(dummy_file, "ter")

lat_min = 43
lat_max = 46
lon_min = -117
lon_max = -111

mask_lon = (cartopy_dom.XLONG >= lon_min) & (cartopy_dom.XLONG <= lon_max)
mask_lat = (cartopy_dom.XLAT >= lat_min) & (cartopy_dom.XLAT <= lat_max)

del dummy_file, cartopy_dom

In [ ]:
QCLOUD_snowie_oct = getvar(wrflist[0:744], "QCLOUD", timeidx=ALL_TIMES, method="cat")
QCLOUD_snowie_oct = QCLOUD_snowie_oct.where(mask_lon & mask_lat, drop=True)

In [ ]:
write_xarray_to_netcdf(QCLOUD_snowie_oct, Saved_Data_Path + "QCLOUD_snowie_oct.nc", 
                       mode = 'w', format = 'NETCDF4', group=None, engine=None, encoding=None)

In [ ]:
del QCLOUD_snowie_oct

In [ ]:
QCLOUD_snowie_nov = getvar(wrflist[744:1464], "QCLOUD", timeidx=ALL_TIMES, method="cat")
QCLOUD_snowie_nov = QCLOUD_snowie_nov.where(mask_lon & mask_lat, drop=True)

In [ ]:
write_xarray_to_netcdf(QCLOUD_snowie_nov, Saved_Data_Path + "QCLOUD_snowie_nov.nc", 
                       mode = 'w', format = 'NETCDF4', group=None, engine=None, encoding=None)

In [ ]:
del QCLOUD_snowie_nov

In [ ]:
QCLOUD_snowie_dec = getvar(wrflist[1464:2208], "QCLOUD", timeidx=ALL_TIMES, method="cat")
QCLOUD_snowie_dec = QCLOUD_snowie_dec.where(mask_lon & mask_lat, drop=True)

In [ ]:
write_xarray_to_netcdf(QCLOUD_snowie_dec, Saved_Data_Path + "QCLOUD_snowie_dec.nc", 
                       mode = 'w', format = 'NETCDF4', group=None, engine=None, encoding=None)

In [ ]:
del QCLOUD_snowie_dec

In [ ]:
QCLOUD_snowie_jan = getvar(wrflist[2208:2952], "QCLOUD", timeidx=ALL_TIMES, method="cat")
QCLOUD_snowie_jan = QCLOUD_snowie_jan.where(mask_lon & mask_lat, drop=True)

In [ ]:
write_xarray_to_netcdf(QCLOUD_snowie_jan, Saved_Data_Path + "QCLOUD_snowie_jan.nc", 
                       mode = 'w', format = 'NETCDF4', group=None, engine=None, encoding=None)

In [ ]:
del QCLOUD_snowie_jan

In [ ]:
QCLOUD_snowie_feb = getvar(wrflist[2952:3624], "QCLOUD", timeidx=ALL_TIMES, method="cat")
QCLOUD_snowie_feb = QCLOUD_snowie_feb.where(mask_lon & mask_lat, drop=True)

In [ ]:
write_xarray_to_netcdf(QCLOUD_snowie_feb, Saved_Data_Path + "QCLOUD_snowie_feb.nc", 
                       mode = 'w', format = 'NETCDF4', group=None, engine=None, encoding=None)

In [ ]:
del QCLOUD_snowie_feb

In [ ]:
QCLOUD_snowie_mar = getvar(wrflist[3624:4368], "QCLOUD", timeidx=ALL_TIMES, method="cat")
QCLOUD_snowie_mar = QCLOUD_snowie_mar.where(mask_lon & mask_lat, drop=True)

In [ ]:
write_xarray_to_netcdf(QCLOUD_snowie_mar, Saved_Data_Path + "QCLOUD_snowie_mar.nc", 
                       mode = 'w', format = 'NETCDF4', group=None, engine=None, encoding=None)

In [ ]:
del QCLOUD_snowie_mar

In [ ]:
QCLOUD_snowie_apr = getvar(wrflist[4368:5088], "QCLOUD", timeidx=ALL_TIMES, method="cat")
QCLOUD_snowie_apr = QCLOUD_snowie_apr.where(mask_lon & mask_lat, drop=True)

In [ ]:
write_xarray_to_netcdf(QCLOUD_snowie_apr, Saved_Data_Path + "QCLOUD_snowie_apr.nc", 
                       mode = 'w', format = 'NETCDF4', group=None, engine=None, encoding=None)

In [ ]:
del QCLOUD_snowie_apr

## **Grab CLDFRA Data Month by Month -- Store to Scratch**

**Load Mask Info**

In [ ]:
dummy_file = Dataset(SNOWIE_Data_Path + '/wrfout_d02_2016-10-01_00:00:00')
cartopy_dom = getvar(dummy_file, "ter")

lat_min = 43
lat_max = 46
lon_min = -117
lon_max = -111

mask_lon = (cartopy_dom.XLONG >= lon_min) & (cartopy_dom.XLONG <= lon_max)
mask_lat = (cartopy_dom.XLAT >= lat_min) & (cartopy_dom.XLAT <= lat_max)

del dummy_file, cartopy_dom

In [ ]:
CLDFRA_snowie_oct = getvar(wrflist[0:744], "CLDFRA", timeidx=ALL_TIMES, method="cat")
CLDFRA_snowie_oct = CLDFRA_snowie_oct.where(mask_lon & mask_lat, drop=True)

write_xarray_to_netcdf(CLDFRA_snowie_oct, Saved_Data_Path + "CLDFRA_snowie_oct.nc", 
                       mode = 'w', format = 'NETCDF4', group=None, engine=None, encoding=None)

del CLDFRA_snowie_oct

CLDFRA_snowie_nov = getvar(wrflist[744:1464], "CLDFRA", timeidx=ALL_TIMES, method="cat")
CLDFRA_snowie_nov = CLDFRA_snowie_nov.where(mask_lon & mask_lat, drop=True)

write_xarray_to_netcdf(CLDFRA_snowie_nov, Saved_Data_Path + "CLDFRA_snowie_nov.nc", 
                       mode = 'w', format = 'NETCDF4', group=None, engine=None, encoding=None)

del CLDFRA_snowie_nov

CLDFRA_snowie_dec = getvar(wrflist[1464:2208], "CLDFRA", timeidx=ALL_TIMES, method="cat")
CLDFRA_snowie_dec = CLDFRA_snowie_dec.where(mask_lon & mask_lat, drop=True)

write_xarray_to_netcdf(CLDFRA_snowie_dec, Saved_Data_Path + "CLDFRA_snowie_dec.nc", 
                       mode = 'w', format = 'NETCDF4', group=None, engine=None, encoding=None)

del CLDFRA_snowie_dec

CLDFRA_snowie_jan = getvar(wrflist[2208:2952], "CLDFRA", timeidx=ALL_TIMES, method="cat")
CLDFRA_snowie_jan = CLDFRA_snowie_jan.where(mask_lon & mask_lat, drop=True)

write_xarray_to_netcdf(CLDFRA_snowie_jan, Saved_Data_Path + "CLDFRA_snowie_jan.nc", 
                       mode = 'w', format = 'NETCDF4', group=None, engine=None, encoding=None)

del CLDFRA_snowie_jan

CLDFRA_snowie_feb = getvar(wrflist[2952:3624], "CLDFRA", timeidx=ALL_TIMES, method="cat")
CLDFRA_snowie_feb = CLDFRA_snowie_feb.where(mask_lon & mask_lat, drop=True)

write_xarray_to_netcdf(CLDFRA_snowie_feb, Saved_Data_Path + "CLDFRA_snowie_feb.nc", 
                       mode = 'w', format = 'NETCDF4', group=None, engine=None, encoding=None)

del CLDFRA_snowie_feb

CLDFRA_snowie_mar = getvar(wrflist[3624:4368], "CLDFRA", timeidx=ALL_TIMES, method="cat")
CLDFRA_snowie_mar = CLDFRA_snowie_mar.where(mask_lon & mask_lat, drop=True)

write_xarray_to_netcdf(CLDFRA_snowie_mar, Saved_Data_Path + "CLDFRA_snowie_mar.nc", 
                       mode = 'w', format = 'NETCDF4', group=None, engine=None, encoding=None)

del CLDFRA_snowie_mar

CLDFRA_snowie_apr = getvar(wrflist[4368:5088], "CLDFRA", timeidx=ALL_TIMES, method="cat")
CLDFRA_snowie_apr = CLDFRA_snowie_apr.where(mask_lon & mask_lat, drop=True)

write_xarray_to_netcdf(CLDFRA_snowie_apr, Saved_Data_Path + "CLDFRA_snowie_apr.nc", 
                       mode = 'w', format = 'NETCDF4', group=None, engine=None, encoding=None)

del CLDFRA_snowie_apr

## **Grab QSNOW Data Month by Month -- Store to Scratch**

**Load Mask Info**

In [ ]:
dummy_file = Dataset(SNOWIE_Data_Path + '/wrfout_d02_2016-10-01_00:00:00')
cartopy_dom = getvar(dummy_file, "ter")

lat_min = 43
lat_max = 46
lon_min = -117
lon_max = -111

mask_lon = (cartopy_dom.XLONG >= lon_min) & (cartopy_dom.XLONG <= lon_max)
mask_lat = (cartopy_dom.XLAT >= lat_min) & (cartopy_dom.XLAT <= lat_max)

del dummy_file, cartopy_dom

In [ ]:
QSNOW_snowie_oct = getvar(wrflist[0:744], "QSNOW", timeidx=ALL_TIMES, method="cat")
QSNOW_snowie_oct = QSNOW_snowie_oct.where(mask_lon & mask_lat, drop=True)

write_xarray_to_netcdf(QSNOW_snowie_oct, Saved_Data_Path + "QSNOW_snowie_oct.nc", 
                       mode = 'w', format = 'NETCDF4', group=None, engine=None, encoding=None)

del QSNOW_snowie_oct

QSNOW_snowie_nov = getvar(wrflist[744:1464], "QSNOW", timeidx=ALL_TIMES, method="cat")
QSNOW_snowie_nov = QSNOW_snowie_nov.where(mask_lon & mask_lat, drop=True)

write_xarray_to_netcdf(QSNOW_snowie_nov, Saved_Data_Path + "QSNOW_snowie_nov.nc", 
                       mode = 'w', format = 'NETCDF4', group=None, engine=None, encoding=None)

del QSNOW_snowie_nov

QSNOW_snowie_dec = getvar(wrflist[1464:2208], "QSNOW", timeidx=ALL_TIMES, method="cat")
QSNOW_snowie_dec = QSNOW_snowie_dec.where(mask_lon & mask_lat, drop=True)

write_xarray_to_netcdf(QSNOW_snowie_dec, Saved_Data_Path + "QSNOW_snowie_dec.nc", 
                       mode = 'w', format = 'NETCDF4', group=None, engine=None, encoding=None)

del QSNOW_snowie_dec

QSNOW_snowie_jan = getvar(wrflist[2208:2952], "QSNOW", timeidx=ALL_TIMES, method="cat")
QSNOW_snowie_jan = QSNOW_snowie_jan.where(mask_lon & mask_lat, drop=True)

write_xarray_to_netcdf(QSNOW_snowie_jan, Saved_Data_Path + "QSNOW_snowie_jan.nc", 
                       mode = 'w', format = 'NETCDF4', group=None, engine=None, encoding=None)

del QSNOW_snowie_jan

QSNOW_snowie_feb = getvar(wrflist[2952:3624], "QSNOW", timeidx=ALL_TIMES, method="cat")
QSNOW_snowie_feb = QSNOW_snowie_feb.where(mask_lon & mask_lat, drop=True)

write_xarray_to_netcdf(QSNOW_snowie_feb, Saved_Data_Path + "QSNOW_snowie_feb.nc", 
                       mode = 'w', format = 'NETCDF4', group=None, engine=None, encoding=None)

del QSNOW_snowie_feb

QSNOW_snowie_mar = getvar(wrflist[3624:4368], "QSNOW", timeidx=ALL_TIMES, method="cat")
QSNOW_snowie_mar = QSNOW_snowie_mar.where(mask_lon & mask_lat, drop=True)

write_xarray_to_netcdf(QSNOW_snowie_mar, Saved_Data_Path + "QSNOW_snowie_mar.nc", 
                       mode = 'w', format = 'NETCDF4', group=None, engine=None, encoding=None)

del QSNOW_snowie_mar

QSNOW_snowie_apr = getvar(wrflist[4368:5088], "QSNOW", timeidx=ALL_TIMES, method="cat")
QSNOW_snowie_apr = QSNOW_snowie_apr.where(mask_lon & mask_lat, drop=True)

write_xarray_to_netcdf(QSNOW_snowie_apr, Saved_Data_Path + "QSNOW_snowie_apr.nc", 
                       mode = 'w', format = 'NETCDF4', group=None, engine=None, encoding=None)

del QSNOW_snowie_apr

## **Grab Z Data Month by Month -- Store to Scratch**

**Load Mask Info**

In [ ]:
dummy_file = Dataset(SNOWIE_Data_Path + '/wrfout_d02_2016-10-01_00:00:00')
cartopy_dom = getvar(dummy_file, "ter")

lat_min = 43
lat_max = 46
lon_min = -117
lon_max = -111

mask_lon = (cartopy_dom.XLONG >= lon_min) & (cartopy_dom.XLONG <= lon_max)
mask_lat = (cartopy_dom.XLAT >= lat_min) & (cartopy_dom.XLAT <= lat_max)

del dummy_file, cartopy_dom

In [ ]:
# z_snowie_oct = getvar(wrflist[0:744], "z", timeidx=ALL_TIMES, method="cat")
# z_snowie_oct = z_snowie_oct.where(mask_lon & mask_lat, drop=True)

# write_xarray_to_netcdf(z_snowie_oct, Saved_Data_Path + "z_snowie_oct.nc", 
#                        mode = 'w', format = 'NETCDF4', group=None, engine=None, encoding=None)

# del z_snowie_oct

# z_snowie_nov = getvar(wrflist[744:1464], "z", timeidx=ALL_TIMES, method="cat")
# z_snowie_nov = z_snowie_nov.where(mask_lon & mask_lat, drop=True)

# write_xarray_to_netcdf(z_snowie_nov, Saved_Data_Path + "z_snowie_nov.nc", 
#                        mode = 'w', format = 'NETCDF4', group=None, engine=None, encoding=None)

# del z_snowie_nov

z_snowie_dec = getvar(wrflist[1464:2208], "z", timeidx=ALL_TIMES, method="cat")
z_snowie_dec = z_snowie_dec.where(mask_lon & mask_lat, drop=True)

write_xarray_to_netcdf(z_snowie_dec, Saved_Data_Path + "z_snowie_dec.nc", 
                       mode = 'w', format = 'NETCDF4', group=None, engine=None, encoding=None)

del z_snowie_dec

z_snowie_jan = getvar(wrflist[2208:2952], "z", timeidx=ALL_TIMES, method="cat")
z_snowie_jan = z_snowie_jan.where(mask_lon & mask_lat, drop=True)

write_xarray_to_netcdf(z_snowie_jan, Saved_Data_Path + "z_snowie_jan.nc", 
                       mode = 'w', format = 'NETCDF4', group=None, engine=None, encoding=None)

del z_snowie_jan

z_snowie_feb = getvar(wrflist[2952:3624], "z", timeidx=ALL_TIMES, method="cat")
z_snowie_feb = z_snowie_feb.where(mask_lon & mask_lat, drop=True)

write_xarray_to_netcdf(z_snowie_feb, Saved_Data_Path + "z_snowie_feb.nc", 
                       mode = 'w', format = 'NETCDF4', group=None, engine=None, encoding=None)

del z_snowie_feb

z_snowie_mar = getvar(wrflist[3624:4368], "z", timeidx=ALL_TIMES, method="cat")
z_snowie_mar = z_snowie_mar.where(mask_lon & mask_lat, drop=True)

write_xarray_to_netcdf(z_snowie_mar, Saved_Data_Path + "z_snowie_mar.nc", 
                       mode = 'w', format = 'NETCDF4', group=None, engine=None, encoding=None)

del z_snowie_mar

z_snowie_apr = getvar(wrflist[4368:5088], "z", timeidx=ALL_TIMES, method="cat")
z_snowie_apr = z_snowie_apr.where(mask_lon & mask_lat, drop=True)

write_xarray_to_netcdf(z_snowie_apr, Saved_Data_Path + "z_snowie_apr.nc", 
                       mode = 'w', format = 'NETCDF4', group=None, engine=None, encoding=None)

del z_snowie_apr

## **Grab Temperature Data Month by Month -- Store to Scratch**

In [ ]:
dummy_file = Dataset(SNOWIE_Data_Path + '/wrfout_d02_2016-10-01_00:00:00')
cartopy_dom = getvar(dummy_file, "ter")

lat_min = 43
lat_max = 46
lon_min = -117
lon_max = -111

mask_lon = (cartopy_dom.XLONG >= lon_min) & (cartopy_dom.XLONG <= lon_max)
mask_lat = (cartopy_dom.XLAT >= lat_min) & (cartopy_dom.XLAT <= lat_max)

del dummy_file, cartopy_dom

In [ ]:
temp_snowie_oct = getvar(wrflist[0:744], "temp", timeidx=ALL_TIMES, method="cat")
temp_snowie_oct = temp_snowie_oct.where(mask_lon & mask_lat, drop=True)

write_xarray_to_netcdf(temp_snowie_oct, Saved_Data_Path + "temp_snowie_oct.nc", 
                       mode = 'w', format = 'NETCDF4', group=None, engine=None, encoding=None)

del temp_snowie_oct

temp_snowie_nov = getvar(wrflist[744:1464], "temp", timeidx=ALL_TIMES, method="cat")
temp_snowie_nov = temp_snowie_nov.where(mask_lon & mask_lat, drop=True)

write_xarray_to_netcdf(temp_snowie_nov, Saved_Data_Path + "temp_snowie_nov.nc", 
                       mode = 'w', format = 'NETCDF4', group=None, engine=None, encoding=None)

del temp_snowie_nov

temp_snowie_dec = getvar(wrflist[1464:2208], "temp", timeidx=ALL_TIMES, method="cat")
temp_snowie_dec = temp_snowie_dec.where(mask_lon & mask_lat, drop=True)

write_xarray_to_netcdf(temp_snowie_dec, Saved_Data_Path + "temp_snowie_dec.nc", 
                       mode = 'w', format = 'NETCDF4', group=None, engine=None, encoding=None)

del temp_snowie_dec

temp_snowie_jan = getvar(wrflist[2208:2952], "temp", timeidx=ALL_TIMES, method="cat")
temp_snowie_jan = temp_snowie_jan.where(mask_lon & mask_lat, drop=True)

write_xarray_to_netcdf(temp_snowie_jan, Saved_Data_Path + "temp_snowie_jan.nc", 
                       mode = 'w', format = 'NETCDF4', group=None, engine=None, encoding=None)

del temp_snowie_jan

temp_snowie_feb = getvar(wrflist[2952:3624], "temp", timeidx=ALL_TIMES, method="cat")
temp_snowie_feb = temp_snowie_feb.where(mask_lon & mask_lat, drop=True)

write_xarray_to_netcdf(temp_snowie_feb, Saved_Data_Path + "temp_snowie_feb.nc", 
                       mode = 'w', format = 'NETCDF4', group=None, engine=None, encoding=None)

del temp_snowie_feb

temp_snowie_mar = getvar(wrflist[3624:4368], "temp", timeidx=ALL_TIMES, method="cat")
temp_snowie_mar = temp_snowie_mar.where(mask_lon & mask_lat, drop=True)

write_xarray_to_netcdf(temp_snowie_mar, Saved_Data_Path + "temp_snowie_mar.nc", 
                       mode = 'w', format = 'NETCDF4', group=None, engine=None, encoding=None)

del temp_snowie_mar

temp_snowie_apr = getvar(wrflist[4368:5088], "temp", timeidx=ALL_TIMES, method="cat")
temp_snowie_apr = temp_snowie_apr.where(mask_lon & mask_lat, drop=True)

write_xarray_to_netcdf(temp_snowie_apr, Saved_Data_Path + "temp_snowie_apr.nc", 
                       mode = 'w', format = 'NETCDF4', group=None, engine=None, encoding=None)

del temp_snowie_apr

## **Grab QRAIN Data Month by Month -- Store to Scratch**

In [ ]:
dummy_file = Dataset(SNOWIE_Data_Path + '/wrfout_d02_2016-10-01_00:00:00')
cartopy_dom = getvar(dummy_file, "ter")

lat_min = 43
lat_max = 46
lon_min = -117
lon_max = -111

mask_lon = (cartopy_dom.XLONG >= lon_min) & (cartopy_dom.XLONG <= lon_max)
mask_lat = (cartopy_dom.XLAT >= lat_min) & (cartopy_dom.XLAT <= lat_max)

del dummy_file, cartopy_dom

In [ ]:
QRAIN_snowie_oct = getvar(wrflist[0:744], "QRAIN", timeidx=ALL_TIMES, method="cat")
QRAIN_snowie_oct = QRAIN_snowie_oct.where(mask_lon & mask_lat, drop=True)

write_xarray_to_netcdf(QRAIN_snowie_oct, Saved_Data_Path + "QRAIN_snowie_oct.nc", 
                       mode = 'w', format = 'NETCDF4', group=None, engine=None, encoding=None)

del QRAIN_snowie_oct

QRAIN_snowie_nov = getvar(wrflist[744:1464], "QRAIN", timeidx=ALL_TIMES, method="cat")
QRAIN_snowie_nov = QRAIN_snowie_nov.where(mask_lon & mask_lat, drop=True)

write_xarray_to_netcdf(QRAIN_snowie_nov, Saved_Data_Path + "QRAIN_snowie_nov.nc", 
                       mode = 'w', format = 'NETCDF4', group=None, engine=None, encoding=None)

del QRAIN_snowie_nov

QRAIN_snowie_dec = getvar(wrflist[1464:2208], "QRAIN", timeidx=ALL_TIMES, method="cat")
QRAIN_snowie_dec = QRAIN_snowie_dec.where(mask_lon & mask_lat, drop=True)

write_xarray_to_netcdf(QRAIN_snowie_dec, Saved_Data_Path + "QRAIN_snowie_dec.nc", 
                       mode = 'w', format = 'NETCDF4', group=None, engine=None, encoding=None)

del QRAIN_snowie_dec

QRAIN_snowie_jan = getvar(wrflist[2208:2952], "QRAIN", timeidx=ALL_TIMES, method="cat")
QRAIN_snowie_jan = QRAIN_snowie_jan.where(mask_lon & mask_lat, drop=True)

write_xarray_to_netcdf(QRAIN_snowie_jan, Saved_Data_Path + "QRAIN_snowie_jan.nc", 
                       mode = 'w', format = 'NETCDF4', group=None, engine=None, encoding=None)

del QRAIN_snowie_jan

QRAIN_snowie_feb = getvar(wrflist[2952:3624], "QRAIN", timeidx=ALL_TIMES, method="cat")
QRAIN_snowie_feb = QRAIN_snowie_feb.where(mask_lon & mask_lat, drop=True)

write_xarray_to_netcdf(QRAIN_snowie_feb, Saved_Data_Path + "QRAIN_snowie_feb.nc", 
                       mode = 'w', format = 'NETCDF4', group=None, engine=None, encoding=None)

del QRAIN_snowie_feb

QRAIN_snowie_mar = getvar(wrflist[3624:4368], "QRAIN", timeidx=ALL_TIMES, method="cat")
QRAIN_snowie_mar = QRAIN_snowie_mar.where(mask_lon & mask_lat, drop=True)

write_xarray_to_netcdf(QRAIN_snowie_mar, Saved_Data_Path + "QRAIN_snowie_mar.nc", 
                       mode = 'w', format = 'NETCDF4', group=None, engine=None, encoding=None)

del QRAIN_snowie_mar

QRAIN_snowie_apr = getvar(wrflist[4368:5088], "QRAIN", timeidx=ALL_TIMES, method="cat")
QRAIN_snowie_apr = QRAIN_snowie_apr.where(mask_lon & mask_lat, drop=True)

write_xarray_to_netcdf(QRAIN_snowie_apr, Saved_Data_Path + "QRAIN_snowie_apr.nc", 
                       mode = 'w', format = 'NETCDF4', group=None, engine=None, encoding=None)

del QRAIN_snowie_apr

## **Grab QGRAUPEL Data Month by Month -- Store to Scratch**

In [ ]:
dummy_file = Dataset(SNOWIE_Data_Path + '/wrfout_d02_2016-10-01_00:00:00')
cartopy_dom = getvar(dummy_file, "ter")

lat_min = 43
lat_max = 46
lon_min = -117
lon_max = -111

mask_lon = (cartopy_dom.XLONG >= lon_min) & (cartopy_dom.XLONG <= lon_max)
mask_lat = (cartopy_dom.XLAT >= lat_min) & (cartopy_dom.XLAT <= lat_max)

del dummy_file, cartopy_dom

In [ ]:
QGRAUP_snowie_oct = getvar(wrflist[0:744], "QGRAUP", timeidx=ALL_TIMES, method="cat")
QGRAUP_snowie_oct = QGRAUP_snowie_oct.where(mask_lon & mask_lat, drop=True)

write_xarray_to_netcdf(QGRAUP_snowie_oct, Saved_Data_Path + "QGRAUP_snowie_oct.nc", 
                       mode = 'w', format = 'NETCDF4', group=None, engine=None, encoding=None)

del QGRAUP_snowie_oct

QGRAUP_snowie_nov = getvar(wrflist[744:1464], "QGRAUP", timeidx=ALL_TIMES, method="cat")
QGRAUP_snowie_nov = QGRAUP_snowie_nov.where(mask_lon & mask_lat, drop=True)

write_xarray_to_netcdf(QGRAUP_snowie_nov, Saved_Data_Path + "QGRAUP_snowie_nov.nc", 
                       mode = 'w', format = 'NETCDF4', group=None, engine=None, encoding=None)

del QGRAUP_snowie_nov

QGRAUP_snowie_dec = getvar(wrflist[1464:2208], "QGRAUP", timeidx=ALL_TIMES, method="cat")
QGRAUP_snowie_dec = QGRAUP_snowie_dec.where(mask_lon & mask_lat, drop=True)

write_xarray_to_netcdf(QGRAUP_snowie_dec, Saved_Data_Path + "QGRAUP_snowie_dec.nc", 
                       mode = 'w', format = 'NETCDF4', group=None, engine=None, encoding=None)

del QGRAUP_snowie_dec

QGRAUP_snowie_jan = getvar(wrflist[2208:2952], "QGRAUP", timeidx=ALL_TIMES, method="cat")
QGRAUP_snowie_jan = QGRAUP_snowie_jan.where(mask_lon & mask_lat, drop=True)

write_xarray_to_netcdf(QGRAUP_snowie_jan, Saved_Data_Path + "QGRAUP_snowie_jan.nc", 
                       mode = 'w', format = 'NETCDF4', group=None, engine=None, encoding=None)

del QGRAUP_snowie_jan

QGRAUP_snowie_feb = getvar(wrflist[2952:3624], "QGRAUP", timeidx=ALL_TIMES, method="cat")
QGRAUP_snowie_feb = QGRAUP_snowie_feb.where(mask_lon & mask_lat, drop=True)

write_xarray_to_netcdf(QGRAUP_snowie_feb, Saved_Data_Path + "QGRAUP_snowie_feb.nc", 
                       mode = 'w', format = 'NETCDF4', group=None, engine=None, encoding=None)

del QGRAUP_snowie_feb

QGRAUP_snowie_mar = getvar(wrflist[3624:4368], "QGRAUP", timeidx=ALL_TIMES, method="cat")
QGRAUP_snowie_mar = QGRAUP_snowie_mar.where(mask_lon & mask_lat, drop=True)

write_xarray_to_netcdf(QGRAUP_snowie_mar, Saved_Data_Path + "QGRAUP_snowie_mar.nc", 
                       mode = 'w', format = 'NETCDF4', group=None, engine=None, encoding=None)

del QGRAUP_snowie_mar

QGRAUP_snowie_apr = getvar(wrflist[4368:5088], "QGRAUP", timeidx=ALL_TIMES, method="cat")
QGRAUP_snowie_apr = QGRAUP_snowie_apr.where(mask_lon & mask_lat, drop=True)

write_xarray_to_netcdf(QGRAUP_snowie_apr, Saved_Data_Path + "QGRAUP_snowie_apr.nc", 
                       mode = 'w', format = 'NETCDF4', group=None, engine=None, encoding=None)

del QGRAUP_snowie_apr

## **Grab QICE Data Month by Month -- Store to Scratch**

In [ ]:
dummy_file = Dataset(SNOWIE_Data_Path + '/wrfout_d02_2016-10-01_00:00:00')
cartopy_dom = getvar(dummy_file, "ter")

lat_min = 43
lat_max = 46
lon_min = -117
lon_max = -111

mask_lon = (cartopy_dom.XLONG >= lon_min) & (cartopy_dom.XLONG <= lon_max)
mask_lat = (cartopy_dom.XLAT >= lat_min) & (cartopy_dom.XLAT <= lat_max)

del dummy_file, cartopy_dom

In [ ]:
QICE_snowie_oct = getvar(wrflist[0:744], "QICE", timeidx=ALL_TIMES, method="cat")
QICE_snowie_oct = QICE_snowie_oct.where(mask_lon & mask_lat, drop=True)

write_xarray_to_netcdf(QICE_snowie_oct, Saved_Data_Path + "QICE_snowie_oct.nc", 
                       mode = 'w', format = 'NETCDF4', group=None, engine=None, encoding=None)

del QICE_snowie_oct

QICE_snowie_nov = getvar(wrflist[744:1464], "QICE", timeidx=ALL_TIMES, method="cat")
QICE_snowie_nov = QICE_snowie_nov.where(mask_lon & mask_lat, drop=True)

write_xarray_to_netcdf(QICE_snowie_nov, Saved_Data_Path + "QICE_snowie_nov.nc", 
                       mode = 'w', format = 'NETCDF4', group=None, engine=None, encoding=None)

del QICE_snowie_nov

QICE_snowie_dec = getvar(wrflist[1464:2208], "QICE", timeidx=ALL_TIMES, method="cat")
QICE_snowie_dec = QICE_snowie_dec.where(mask_lon & mask_lat, drop=True)

write_xarray_to_netcdf(QICE_snowie_dec, Saved_Data_Path + "QICE_snowie_dec.nc", 
                       mode = 'w', format = 'NETCDF4', group=None, engine=None, encoding=None)

del QICE_snowie_dec

QICE_snowie_jan = getvar(wrflist[2208:2952], "QICE", timeidx=ALL_TIMES, method="cat")
QICE_snowie_jan = QICE_snowie_jan.where(mask_lon & mask_lat, drop=True)

write_xarray_to_netcdf(QICE_snowie_jan, Saved_Data_Path + "QICE_snowie_jan.nc", 
                       mode = 'w', format = 'NETCDF4', group=None, engine=None, encoding=None)

del QICE_snowie_jan

QICE_snowie_feb = getvar(wrflist[2952:3624], "QICE", timeidx=ALL_TIMES, method="cat")
QICE_snowie_feb = QICE_snowie_feb.where(mask_lon & mask_lat, drop=True)

write_xarray_to_netcdf(QICE_snowie_feb, Saved_Data_Path + "QICE_snowie_feb.nc", 
                       mode = 'w', format = 'NETCDF4', group=None, engine=None, encoding=None)

del QICE_snowie_feb

QICE_snowie_mar = getvar(wrflist[3624:4368], "QICE", timeidx=ALL_TIMES, method="cat")
QICE_snowie_mar = QICE_snowie_mar.where(mask_lon & mask_lat, drop=True)

write_xarray_to_netcdf(QICE_snowie_mar, Saved_Data_Path + "QICE_snowie_mar.nc", 
                       mode = 'w', format = 'NETCDF4', group=None, engine=None, encoding=None)

del QICE_snowie_mar

QICE_snowie_apr = getvar(wrflist[4368:5088], "QICE", timeidx=ALL_TIMES, method="cat")
QICE_snowie_apr = QICE_snowie_apr.where(mask_lon & mask_lat, drop=True)

write_xarray_to_netcdf(QICE_snowie_apr, Saved_Data_Path + "QICE_snowie_apr.nc", 
                       mode = 'w', format = 'NETCDF4', group=None, engine=None, encoding=None)

del QICE_snowie_apr

## **Grab Theta_E Data Month by Month -- Store to Scratch**

In [5]:
dummy_file = Dataset(SNOWIE_Data_Path + '/wrfout_d02_2016-10-01_00:00:00')
cartopy_dom = getvar(dummy_file, "ter")

lat_min = 43
lat_max = 46
lon_min = -117
lon_max = -111

mask_lon = (cartopy_dom.XLONG >= lon_min) & (cartopy_dom.XLONG <= lon_max)
mask_lat = (cartopy_dom.XLAT >= lat_min) & (cartopy_dom.XLAT <= lat_max)

del dummy_file, cartopy_dom

In [ ]:
ThetaE_snowie_oct = getvar(wrflist[0:744], "theta_e", timeidx=ALL_TIMES, method="cat")
ThetaE_snowie_oct = ThetaE_snowie_oct.where(mask_lon & mask_lat, drop=True)

write_xarray_to_netcdf(ThetaE_snowie_oct, Saved_Data_Path + "ThetaE_snowie_oct.nc", 
                       mode = 'w', format = 'NETCDF4', group=None, engine=None, encoding=None)

del ThetaE_snowie_oct
print("1 done")

ThetaE_snowie_nov = getvar(wrflist[744:1464], "theta_e", timeidx=ALL_TIMES, method="cat")
ThetaE_snowie_nov = ThetaE_snowie_nov.where(mask_lon & mask_lat, drop=True)

write_xarray_to_netcdf(ThetaE_snowie_nov, Saved_Data_Path + "ThetaE_snowie_nov.nc", 
                       mode = 'w', format = 'NETCDF4', group=None, engine=None, encoding=None)

del ThetaE_snowie_nov
print("2 done")

ThetaE_snowie_dec = getvar(wrflist[1464:2208], "theta_e", timeidx=ALL_TIMES, method="cat")
ThetaE_snowie_dec = ThetaE_snowie_dec.where(mask_lon & mask_lat, drop=True)

write_xarray_to_netcdf(ThetaE_snowie_dec, Saved_Data_Path + "ThetaE_snowie_dec.nc", 
                       mode = 'w', format = 'NETCDF4', group=None, engine=None, encoding=None)

del ThetaE_snowie_dec
print("3 done")

ThetaE_snowie_jan = getvar(wrflist[2208:2952], "theta_e", timeidx=ALL_TIMES, method="cat")
ThetaE_snowie_jan = ThetaE_snowie_jan.where(mask_lon & mask_lat, drop=True)

write_xarray_to_netcdf(ThetaE_snowie_jan, Saved_Data_Path + "ThetaE_snowie_jan.nc", 
                       mode = 'w', format = 'NETCDF4', group=None, engine=None, encoding=None)

del ThetaE_snowie_jan
print("4 done")

ThetaE_snowie_feb = getvar(wrflist[2952:3624], "theta_e", timeidx=ALL_TIMES, method="cat")
ThetaE_snowie_feb = ThetaE_snowie_feb.where(mask_lon & mask_lat, drop=True)

write_xarray_to_netcdf(ThetaE_snowie_feb, Saved_Data_Path + "ThetaE_snowie_feb.nc", 
                       mode = 'w', format = 'NETCDF4', group=None, engine=None, encoding=None)

del ThetaE_snowie_feb
print("5 done")

ThetaE_snowie_mar = getvar(wrflist[3624:4368], "theta_e", timeidx=ALL_TIMES, method="cat")
ThetaE_snowie_mar = ThetaE_snowie_mar.where(mask_lon & mask_lat, drop=True)

write_xarray_to_netcdf(ThetaE_snowie_mar, Saved_Data_Path + "ThetaE_snowie_mar.nc", 
                       mode = 'w', format = 'NETCDF4', group=None, engine=None, encoding=None)

del ThetaE_snowie_mar
print("6 done")

ThetaE_snowie_apr = getvar(wrflist[4368:5088], "theta_e", timeidx=ALL_TIMES, method="cat")
ThetaE_snowie_apr = ThetaE_snowie_apr.where(mask_lon & mask_lat, drop=True)

write_xarray_to_netcdf(ThetaE_snowie_apr, Saved_Data_Path + "ThetaE_snowie_apr.nc", 
                       mode = 'w', format = 'NETCDF4', group=None, engine=None, encoding=None)

del ThetaE_snowie_apr
print("7 done")

1 done
2 done


## **Grab QVAPOR Data Month by Month -- Store to Scratch**

In [5]:
dummy_file = Dataset(SNOWIE_Data_Path + '/wrfout_d02_2016-10-01_00:00:00')
cartopy_dom = getvar(dummy_file, "ter")

lat_min = 43
lat_max = 46
lon_min = -117
lon_max = -111

mask_lon = (cartopy_dom.XLONG >= lon_min) & (cartopy_dom.XLONG <= lon_max)
mask_lat = (cartopy_dom.XLAT >= lat_min) & (cartopy_dom.XLAT <= lat_max)

del dummy_file, cartopy_dom

In [6]:
QVAPOR_snowie_oct = getvar(wrflist[0:744], "QVAPOR", timeidx=ALL_TIMES, method="cat")
QVAPOR_snowie_oct = QVAPOR_snowie_oct.where(mask_lon & mask_lat, drop=True)

write_xarray_to_netcdf(QVAPOR_snowie_oct, Saved_Data_Path + "QVAPOR_snowie_oct.nc", 
                       mode = 'w', format = 'NETCDF4', group=None, engine=None, encoding=None)

del QVAPOR_snowie_oct
print("1 done")

QVAPOR_snowie_nov = getvar(wrflist[744:1464], "QVAPOR", timeidx=ALL_TIMES, method="cat")
QVAPOR_snowie_nov = QVAPOR_snowie_nov.where(mask_lon & mask_lat, drop=True)

write_xarray_to_netcdf(QVAPOR_snowie_nov, Saved_Data_Path + "QVAPOR_snowie_nov.nc", 
                       mode = 'w', format = 'NETCDF4', group=None, engine=None, encoding=None)

del QVAPOR_snowie_nov
print("2 done")

QVAPOR_snowie_dec = getvar(wrflist[1464:2208], "QVAPOR", timeidx=ALL_TIMES, method="cat")
QVAPOR_snowie_dec = QVAPOR_snowie_dec.where(mask_lon & mask_lat, drop=True)

write_xarray_to_netcdf(QVAPOR_snowie_dec, Saved_Data_Path + "QVAPOR_snowie_dec.nc", 
                       mode = 'w', format = 'NETCDF4', group=None, engine=None, encoding=None)

del QVAPOR_snowie_dec
print("3 done")

QVAPOR_snowie_jan = getvar(wrflist[2208:2952], "QVAPOR", timeidx=ALL_TIMES, method="cat")
QVAPOR_snowie_jan = QVAPOR_snowie_jan.where(mask_lon & mask_lat, drop=True)

write_xarray_to_netcdf(QVAPOR_snowie_jan, Saved_Data_Path + "QVAPOR_snowie_jan.nc", 
                       mode = 'w', format = 'NETCDF4', group=None, engine=None, encoding=None)

del QVAPOR_snowie_jan
print("4 done")

QVAPOR_snowie_feb = getvar(wrflist[2952:3624], "QVAPOR", timeidx=ALL_TIMES, method="cat")
QVAPOR_snowie_feb = QVAPOR_snowie_feb.where(mask_lon & mask_lat, drop=True)

write_xarray_to_netcdf(QVAPOR_snowie_feb, Saved_Data_Path + "QVAPOR_snowie_feb.nc", 
                       mode = 'w', format = 'NETCDF4', group=None, engine=None, encoding=None)

del QVAPOR_snowie_feb
print("5 done")

QVAPOR_snowie_mar = getvar(wrflist[3624:4368], "QVAPOR", timeidx=ALL_TIMES, method="cat")
QVAPOR_snowie_mar = QVAPOR_snowie_mar.where(mask_lon & mask_lat, drop=True)

write_xarray_to_netcdf(QVAPOR_snowie_mar, Saved_Data_Path + "QVAPOR_snowie_mar.nc", 
                       mode = 'w', format = 'NETCDF4', group=None, engine=None, encoding=None)

del QVAPOR_snowie_mar
print("6 done")

QVAPOR_snowie_apr = getvar(wrflist[4368:5088], "QVAPOR", timeidx=ALL_TIMES, method="cat")
QVAPOR_snowie_apr = QVAPOR_snowie_apr.where(mask_lon & mask_lat, drop=True)

write_xarray_to_netcdf(QVAPOR_snowie_apr, Saved_Data_Path + "QVAPOR_snowie_apr.nc", 
                       mode = 'w', format = 'NETCDF4', group=None, engine=None, encoding=None)

del QVAPOR_snowie_apr
print("7 done")

1 done
2 done
3 done
4 done
5 done
6 done
7 done


## **Grab u Data Month by Month -- Store to Scratch**

In [5]:
dummy_file = Dataset(SNOWIE_Data_Path + '/wrfout_d02_2016-10-01_00:00:00')
cartopy_dom = getvar(dummy_file, "ter")

lat_min = 43
lat_max = 46
lon_min = -117
lon_max = -111

mask_lon = (cartopy_dom.XLONG >= lon_min) & (cartopy_dom.XLONG <= lon_max)
mask_lat = (cartopy_dom.XLAT >= lat_min) & (cartopy_dom.XLAT <= lat_max)

del dummy_file, cartopy_dom

In [7]:
u_snowie_oct = getvar(wrflist[0:744], "ua", units="m s-1", timeidx=ALL_TIMES, method="cat")
u_snowie_oct = u_snowie_oct.where(mask_lon & mask_lat, drop=True)

write_xarray_to_netcdf(u_snowie_oct, Saved_Data_Path + "u_snowie_oct.nc", 
                       mode = 'w', format = 'NETCDF4', group=None, engine=None, encoding=None)

del u_snowie_oct
print("1 done")

u_snowie_nov = getvar(wrflist[744:1464], "ua", units="m s-1", timeidx=ALL_TIMES, method="cat")
u_snowie_nov = u_snowie_nov.where(mask_lon & mask_lat, drop=True)

write_xarray_to_netcdf(u_snowie_nov, Saved_Data_Path + "u_snowie_nov.nc", 
                       mode = 'w', format = 'NETCDF4', group=None, engine=None, encoding=None)

del u_snowie_nov
print("2 done")

u_snowie_dec = getvar(wrflist[1464:2208], "ua", units="m s-1", timeidx=ALL_TIMES, method="cat")
u_snowie_dec = u_snowie_dec.where(mask_lon & mask_lat, drop=True)

write_xarray_to_netcdf(u_snowie_dec, Saved_Data_Path + "u_snowie_dec.nc", 
                       mode = 'w', format = 'NETCDF4', group=None, engine=None, encoding=None)

del u_snowie_dec
print("3 done")

u_snowie_jan = getvar(wrflist[2208:2952], "ua", units="m s-1", timeidx=ALL_TIMES, method="cat")
u_snowie_jan = u_snowie_jan.where(mask_lon & mask_lat, drop=True)

write_xarray_to_netcdf(u_snowie_jan, Saved_Data_Path + "u_snowie_jan.nc", 
                       mode = 'w', format = 'NETCDF4', group=None, engine=None, encoding=None)

del u_snowie_jan
print("4 done")

u_snowie_feb = getvar(wrflist[2952:3624], "ua", units="m s-1", timeidx=ALL_TIMES, method="cat")
u_snowie_feb = u_snowie_feb.where(mask_lon & mask_lat, drop=True)

write_xarray_to_netcdf(u_snowie_feb, Saved_Data_Path + "u_snowie_feb.nc", 
                       mode = 'w', format = 'NETCDF4', group=None, engine=None, encoding=None)

del u_snowie_feb
print("5 done")

u_snowie_mar = getvar(wrflist[3624:4368], "ua", units="m s-1", timeidx=ALL_TIMES, method="cat")
u_snowie_mar = u_snowie_mar.where(mask_lon & mask_lat, drop=True)

write_xarray_to_netcdf(u_snowie_mar, Saved_Data_Path + "u_snowie_mar.nc", 
                       mode = 'w', format = 'NETCDF4', group=None, engine=None, encoding=None)

del u_snowie_mar
print("6 done")

u_snowie_apr = getvar(wrflist[4368:5088], "ua", units="m s-1", timeidx=ALL_TIMES, method="cat")
u_snowie_apr = u_snowie_apr.where(mask_lon & mask_lat, drop=True)

write_xarray_to_netcdf(u_snowie_apr, Saved_Data_Path + "u_snowie_apr.nc", 
                       mode = 'w', format = 'NETCDF4', group=None, engine=None, encoding=None)

del u_snowie_apr
print("7 done")

1 done
2 done
3 done
4 done
5 done
6 done
7 done


## **Grab v Data Month by Month -- Store to Scratch**

In [5]:
dummy_file = Dataset(SNOWIE_Data_Path + '/wrfout_d02_2016-10-01_00:00:00')
cartopy_dom = getvar(dummy_file, "ter")

lat_min = 43
lat_max = 46
lon_min = -117
lon_max = -111

mask_lon = (cartopy_dom.XLONG >= lon_min) & (cartopy_dom.XLONG <= lon_max)
mask_lat = (cartopy_dom.XLAT >= lat_min) & (cartopy_dom.XLAT <= lat_max)

del dummy_file, cartopy_dom

In [8]:
v_snowie_oct = getvar(wrflist[0:744], "va", units="m s-1", timeidx=ALL_TIMES, method="cat")
v_snowie_oct = v_snowie_oct.where(mask_lon & mask_lat, drop=True)

write_xarray_to_netcdf(v_snowie_oct, Saved_Data_Path + "v_snowie_oct.nc", 
                       mode = 'w', format = 'NETCDF4', group=None, engine=None, encoding=None)

del v_snowie_oct
print("1 done")

v_snowie_nov = getvar(wrflist[744:1464], "va", units="m s-1", timeidx=ALL_TIMES, method="cat")
v_snowie_nov = v_snowie_nov.where(mask_lon & mask_lat, drop=True)

write_xarray_to_netcdf(v_snowie_nov, Saved_Data_Path + "v_snowie_nov.nc", 
                       mode = 'w', format = 'NETCDF4', group=None, engine=None, encoding=None)

del v_snowie_nov
print("2 done")

v_snowie_dec = getvar(wrflist[1464:2208], "va", units="m s-1", timeidx=ALL_TIMES, method="cat")
v_snowie_dec = v_snowie_dec.where(mask_lon & mask_lat, drop=True)

write_xarray_to_netcdf(v_snowie_dec, Saved_Data_Path + "v_snowie_dec.nc", 
                       mode = 'w', format = 'NETCDF4', group=None, engine=None, encoding=None)

del v_snowie_dec
print("3 done")

v_snowie_jan = getvar(wrflist[2208:2952], "va", units="m s-1", timeidx=ALL_TIMES, method="cat")
v_snowie_jan = v_snowie_jan.where(mask_lon & mask_lat, drop=True)

write_xarray_to_netcdf(v_snowie_jan, Saved_Data_Path + "v_snowie_jan.nc", 
                       mode = 'w', format = 'NETCDF4', group=None, engine=None, encoding=None)

del v_snowie_jan
print("4 done")

v_snowie_feb = getvar(wrflist[2952:3624], "va", units="m s-1", timeidx=ALL_TIMES, method="cat")
v_snowie_feb = v_snowie_feb.where(mask_lon & mask_lat, drop=True)

write_xarray_to_netcdf(v_snowie_feb, Saved_Data_Path + "v_snowie_feb.nc", 
                       mode = 'w', format = 'NETCDF4', group=None, engine=None, encoding=None)

del v_snowie_feb
print("5 done")

v_snowie_mar = getvar(wrflist[3624:4368], "va", units="m s-1", timeidx=ALL_TIMES, method="cat")
v_snowie_mar = v_snowie_mar.where(mask_lon & mask_lat, drop=True)

write_xarray_to_netcdf(v_snowie_mar, Saved_Data_Path + "v_snowie_mar.nc", 
                       mode = 'w', format = 'NETCDF4', group=None, engine=None, encoding=None)

del v_snowie_mar
print("6 done")

v_snowie_apr = getvar(wrflist[4368:5088], "va", units="m s-1", timeidx=ALL_TIMES, method="cat")
v_snowie_apr = v_snowie_apr.where(mask_lon & mask_lat, drop=True)

write_xarray_to_netcdf(v_snowie_apr, Saved_Data_Path + "v_snowie_apr.nc", 
                       mode = 'w', format = 'NETCDF4', group=None, engine=None, encoding=None)

del v_snowie_apr
print("7 done")

1 done
2 done
3 done
4 done
5 done
6 done
7 done


## **Grab P Data Month by Month -- Store to Scratch**

In [5]:
dummy_file = Dataset(SNOWIE_Data_Path + '/wrfout_d02_2016-10-01_00:00:00')
cartopy_dom = getvar(dummy_file, "ter")

lat_min = 43
lat_max = 46
lon_min = -117
lon_max = -111

mask_lon = (cartopy_dom.XLONG >= lon_min) & (cartopy_dom.XLONG <= lon_max)
mask_lat = (cartopy_dom.XLAT >= lat_min) & (cartopy_dom.XLAT <= lat_max)

del dummy_file, cartopy_dom

In [10]:
p_snowie_oct = getvar(wrflist[0:744], "pressure", timeidx=ALL_TIMES, method="cat")
p_snowie_oct = p_snowie_oct.where(mask_lon & mask_lat, drop=True)

write_xarray_to_netcdf(p_snowie_oct, Saved_Data_Path + "p_snowie_oct.nc", 
                       mode = 'w', format = 'NETCDF4', group=None, engine=None, encoding=None)

del p_snowie_oct
print("1 done")

p_snowie_nov = getvar(wrflist[744:1464], "pressure", timeidx=ALL_TIMES, method="cat")
p_snowie_nov = p_snowie_nov.where(mask_lon & mask_lat, drop=True)

write_xarray_to_netcdf(p_snowie_nov, Saved_Data_Path + "p_snowie_nov.nc", 
                       mode = 'w', format = 'NETCDF4', group=None, engine=None, encoding=None)

del p_snowie_nov
print("2 done")

p_snowie_dec = getvar(wrflist[1464:2208], "pressure", timeidx=ALL_TIMES, method="cat")
p_snowie_dec = p_snowie_dec.where(mask_lon & mask_lat, drop=True)

write_xarray_to_netcdf(p_snowie_dec, Saved_Data_Path + "p_snowie_dec.nc", 
                       mode = 'w', format = 'NETCDF4', group=None, engine=None, encoding=None)

del p_snowie_dec
print("3 done")

p_snowie_jan = getvar(wrflist[2208:2952], "pressure", timeidx=ALL_TIMES, method="cat")
p_snowie_jan = p_snowie_jan.where(mask_lon & mask_lat, drop=True)

write_xarray_to_netcdf(p_snowie_jan, Saved_Data_Path + "p_snowie_jan.nc", 
                       mode = 'w', format = 'NETCDF4', group=None, engine=None, encoding=None)

del p_snowie_jan
print("4 done")

p_snowie_feb = getvar(wrflist[2952:3624], "pressure", timeidx=ALL_TIMES, method="cat")
p_snowie_feb = p_snowie_feb.where(mask_lon & mask_lat, drop=True)

write_xarray_to_netcdf(p_snowie_feb, Saved_Data_Path + "p_snowie_feb.nc", 
                       mode = 'w', format = 'NETCDF4', group=None, engine=None, encoding=None)

del p_snowie_feb
print("5 done")

p_snowie_mar = getvar(wrflist[3624:4368], "pressure", timeidx=ALL_TIMES, method="cat")
p_snowie_mar = p_snowie_mar.where(mask_lon & mask_lat, drop=True)

write_xarray_to_netcdf(p_snowie_mar, Saved_Data_Path + "p_snowie_mar.nc", 
                       mode = 'w', format = 'NETCDF4', group=None, engine=None, encoding=None)

del p_snowie_mar
print("6 done")

p_snowie_apr = getvar(wrflist[4368:5088], "pressure", timeidx=ALL_TIMES, method="cat")
p_snowie_apr = p_snowie_apr.where(mask_lon & mask_lat, drop=True)

write_xarray_to_netcdf(p_snowie_apr, Saved_Data_Path + "p_snowie_apr.nc", 
                       mode = 'w', format = 'NETCDF4', group=None, engine=None, encoding=None)

del p_snowie_apr
print("7 done")

1 done
2 done
3 done
4 done
5 done
6 done
7 done


## **Grab T_v Data Month by Month -- Store to Scratch**

In [5]:
dummy_file = Dataset(SNOWIE_Data_Path + '/wrfout_d02_2016-10-01_00:00:00')
cartopy_dom = getvar(dummy_file, "ter")

lat_min = 43
lat_max = 46
lon_min = -117
lon_max = -111

mask_lon = (cartopy_dom.XLONG >= lon_min) & (cartopy_dom.XLONG <= lon_max)
mask_lat = (cartopy_dom.XLAT >= lat_min) & (cartopy_dom.XLAT <= lat_max)

del dummy_file, cartopy_dom

In [6]:
# tv_snowie_oct = getvar(wrflist[0:744], "tv", timeidx=ALL_TIMES, method="cat")
# tv_snowie_oct = tv_snowie_oct.where(mask_lon & mask_lat, drop=True)

# write_xarray_to_netcdf(tv_snowie_oct, Saved_Data_Path + "tv_snowie_oct.nc", 
#                        mode = 'w', format = 'NETCDF4', group=None, engine=None, encoding=None)

# del tv_snowie_oct
# print("1 done")

# tv_snowie_nov = getvar(wrflist[744:1464], "tv", timeidx=ALL_TIMES, method="cat")
# tv_snowie_nov = tv_snowie_nov.where(mask_lon & mask_lat, drop=True)

# write_xarray_to_netcdf(tv_snowie_nov, Saved_Data_Path + "tv_snowie_nov.nc", 
#                        mode = 'w', format = 'NETCDF4', group=None, engine=None, encoding=None)

# del tv_snowie_nov
# print("2 done")

# tv_snowie_dec = getvar(wrflist[1464:2208], "tv", timeidx=ALL_TIMES, method="cat")
# tv_snowie_dec = tv_snowie_dec.where(mask_lon & mask_lat, drop=True)

# write_xarray_to_netcdf(tv_snowie_dec, Saved_Data_Path + "tv_snowie_dec.nc", 
#                        mode = 'w', format = 'NETCDF4', group=None, engine=None, encoding=None)

# del tv_snowie_dec
# print("3 done")

# tv_snowie_jan = getvar(wrflist[2208:2952], "tv", timeidx=ALL_TIMES, method="cat")
# tv_snowie_jan = tv_snowie_jan.where(mask_lon & mask_lat, drop=True)

# write_xarray_to_netcdf(tv_snowie_jan, Saved_Data_Path + "tv_snowie_jan.nc", 
#                        mode = 'w', format = 'NETCDF4', group=None, engine=None, encoding=None)

# del tv_snowie_jan
# print("4 done")

# tv_snowie_feb = getvar(wrflist[2952:3624], "tv", timeidx=ALL_TIMES, method="cat")
# tv_snowie_feb = tv_snowie_feb.where(mask_lon & mask_lat, drop=True)

# write_xarray_to_netcdf(tv_snowie_feb, Saved_Data_Path + "tv_snowie_feb.nc", 
#                        mode = 'w', format = 'NETCDF4', group=None, engine=None, encoding=None)

# del tv_snowie_feb
# print("5 done")

tv_snowie_mar = getvar(wrflist[3624:4368], "tv", timeidx=ALL_TIMES, method="cat")
tv_snowie_mar = tv_snowie_mar.where(mask_lon & mask_lat, drop=True)

write_xarray_to_netcdf(tv_snowie_mar, Saved_Data_Path + "tv_snowie_mar.nc", 
                       mode = 'w', format = 'NETCDF4', group=None, engine=None, encoding=None)

del tv_snowie_mar
print("6 done")

tv_snowie_apr = getvar(wrflist[4368:5088], "tv", timeidx=ALL_TIMES, method="cat")
tv_snowie_apr = tv_snowie_apr.where(mask_lon & mask_lat, drop=True)

write_xarray_to_netcdf(tv_snowie_apr, Saved_Data_Path + "tv_snowie_apr.nc", 
                       mode = 'w', format = 'NETCDF4', group=None, engine=None, encoding=None)

del tv_snowie_apr
print("7 done")

6 done
7 done
